In [14]:
import geopandas as gpd
import pandas as pd
from shapely.ops import unary_union
import warnings
warnings.filterwarnings('ignore')

# ===== 設定參數 =====
LAND_USE_PATH = 'p113119-3_valid_geometry_3826.geojson'  # 土地利用資料路徑
STAT_AREA_PATH = 'STAT/113年12月臺北市統計區人口統計_最小統計區/tp_最小統計區.geojson'  # 最小統計區資料路徑
OUTPUT_PATH = 'taipei_open_space_coverage.geojson'  # 輸出路徑
BUFFER_DISTANCE = 300  # 服務範圍 (公尺)

# 定義開放空間類型 (根據你的資料,請依實際狀況調整)
OPEN_SPACE_TYPES = [
    '公園',
    '公園用地',
    '公園綠地',
    '公園廣場用地',
    # '兒童遊樂場',
    # '兒童遊樂場用地',
    # '廣場用地',
    # '綠地',
    # '體育場所用地',
    # '運動場',
    # '學校用地',
    # '社教用地',
    # '人行步道(無遮蔽)',
    # '人行步道'
]

MIN_AREA_THRESHOLD = 1000

print("=" * 60)
print("台北市最小統計區開放空間服務涵蓋率分析")
print("=" * 60)

# ===== Step 1: 讀取資料 =====
print("\n[1/6] 讀取資料...")
land_use = gpd.read_file(LAND_USE_PATH)
stat_areas = gpd.read_file(STAT_AREA_PATH)

print(f"  ✓ 土地利用資料: {len(land_use)} 筆")
print(f"  ✓ 最小統計區: {len(stat_areas)} 筆")
print(f"  ✓ 土地利用座標系統: {land_use.crs}")
print(f"  ✓ 最小統計區座標系統: {stat_areas.crs}")

# ===== Step 2: 座標轉換 =====
print("\n[2/6] 座標系統轉換至 WGS84...")
if land_use.crs != 'EPSG:4326':
    land_use = land_use.to_crs('EPSG:4326')
    print(f"  ✓ 土地利用資料已轉換")

if stat_areas.crs != 'EPSG:4326':
    stat_areas = stat_areas.to_crs('EPSG:4326')
    print(f"  ✓ 最小統計區已轉換")

# 為了計算緩衝區(單位:公尺),暫時轉換至 TWD97 TM2
print("\n[3/6] 建立開放空間服務範圍...")
land_use_meter = land_use.to_crs('EPSG:3826')
stat_areas_meter = stat_areas.to_crs('EPSG:3826')

# ===== Step 3: 篩選開放空間 =====
# 檢查實際的使用分區欄位名稱
land_use_column = '使用分區'  # 根據你的截圖

print(f"\n  土地利用分區類型:")
unique_types = land_use[land_use_column].unique()
print(f"  共 {len(unique_types)} 種分區類型")

# 使用模糊比對找出開放空間
open_spaces_mask = land_use_meter[land_use_column].str.contains(
    # '|'.join(['公園', '綠地', '廣場', '遊樂', '步道', '體育', '運動']), 
    # '|'.join(['公園', '綠地', '廣場', '遊樂']), 
    # '|'.join(['公園', '綠地',  '遊樂']), 
    '|'.join(['公園']), 
    case=False, 
    na=False
)
open_spaces = land_use_meter[open_spaces_mask].copy()
open_spaces.to_crs('EPSG:4326').to_file("taipei_open_spaces.geojson", driver="GeoJSON", encoding="utf-8")

open_spaces = land_use_meter[open_spaces_mask].copy()
print(f"\n  ✓ 篩選出 {len(open_spaces)} 筆開放空間")

if len(open_spaces) > 0:
    print(f"  ✓ 開放空間類型: {open_spaces[land_use_column].unique()[:10]}")




# ===== 面積篩選 =====
print(f"\n  【面積門檻篩選】")
print(f"  設定最小面積門檻: {MIN_AREA_THRESHOLD} 平方公尺")

# 計算每個開放空間的面積
open_spaces['area_sqm'] = open_spaces.geometry.area

# 面積統計
print(f"  篩選前開放空間數量: {len(open_spaces)}")
print(f"  面積統計:")
print(f"    - 最小: {open_spaces['area_sqm'].min():.2f} ㎡")
print(f"    - 最大: {open_spaces['area_sqm'].max():.2f} ㎡")
print(f"    - 平均: {open_spaces['area_sqm'].mean():.2f} ㎡")
print(f"    - 中位數: {open_spaces['area_sqm'].median():.2f} ㎡")

# 篩選符合面積門檻的開放空間
open_spaces_filtered = open_spaces[open_spaces['area_sqm'] >= MIN_AREA_THRESHOLD].copy()

small_spaces_count = len(open_spaces) - len(open_spaces_filtered)
print(f"\n  ✓ 移除 {small_spaces_count} 個過小綠地 (< {MIN_AREA_THRESHOLD}㎡)")
print(f"  ✓ 保留 {len(open_spaces_filtered)} 個有效服務性開放空間")

# 顯示被移除的小綠地面積範圍
if small_spaces_count > 0:
    small_spaces = open_spaces[open_spaces['area_sqm'] < MIN_AREA_THRESHOLD]
    print(f"  ✓ 被移除的小綠地面積範圍: {small_spaces['area_sqm'].min():.0f} - {small_spaces['area_sqm'].max():.0f} ㎡")

# 使用篩選後的開放空間繼續分析
open_spaces = open_spaces_filtered


# ===== Step 4: 建立緩衝區並合併 =====
print(f"\n[4/6] 建立 {BUFFER_DISTANCE} 公尺服務範圍緩衝區...")

# 建立緩衝區
open_spaces['buffer_geom'] = open_spaces.geometry.buffer(BUFFER_DISTANCE)

# 合併所有緩衝區 (關鍵優化!)
print("  ✓ 合併重疊的緩衝區...")
merged_buffer_geom = unary_union(open_spaces['buffer_geom'])

# 建立緩衝區 GeoDataFrame並儲存為GeoJSON
merged_buffer = gpd.GeoDataFrame(
    {'id': [1]},
    geometry=[merged_buffer_geom],
    crs='EPSG:3826'
)
merged_buffer.to_crs('EPSG:4326').to_file("taipei_merged_buffer.geojson", driver="GeoJSON", encoding="utf-8")

# ===== Step 5: 計算涵蓋率 =====
print("\n[5/6] 計算每個統計區的服務涵蓋率...")

# 計算統計區總面積
stat_areas_meter['total_area'] = stat_areas_meter.geometry.area

# 計算服務涵蓋面積
stat_areas_meter['service_geom'] = stat_areas_meter.geometry.intersection(
    merged_buffer_geom
)
stat_areas_meter['service_area'] = stat_areas_meter['service_geom'].area

# 計算涵蓋率 (百分比)
stat_areas_meter['coverage_rate'] = (
    stat_areas_meter['service_area'] / 
    stat_areas_meter['total_area'] * 100
).round(2)

# 分級 (方便視覺化)
stat_areas_meter['coverage_level'] = pd.cut(
    stat_areas_meter['coverage_rate'],
    bins=[0, 25, 50, 75, 100],
    labels=['低(<25%)', '中(25-50%)', '高(50-75%)', '極高(>75%)']
)

print(f"  ✓ 計算完成!")

# ===== Step 6: 轉換回 WGS84 並輸出 =====
print("\n[6/6] 轉換回 WGS84 並儲存結果...")

# 移除臨時幾何欄位,轉換回 WGS84
result = stat_areas_meter.drop(columns=['service_geom']).to_crs('EPSG:4326')

# 儲存結果
result.to_file(OUTPUT_PATH, encoding='utf-8')

print(f"  ✓ 結果已儲存至: {OUTPUT_PATH}")

# ===== 統計摘要 =====
print("\n" + "=" * 60)
print("分析結果摘要")
print("=" * 60)

print(f"\n總統計區數: {len(result)}")
print(f"平均涵蓋率: {result['coverage_rate'].mean():.2f}%")
print(f"中位數涵蓋率: {result['coverage_rate'].median():.2f}%")
print(f"最高涵蓋率: {result['coverage_rate'].max():.2f}%")
print(f"最低涵蓋率: {result['coverage_rate'].min():.2f}%")

print("\n涵蓋率分級統計:")
print(result['coverage_level'].value_counts().sort_index())

print("\n完全無服務的統計區:")
no_service = result[result['coverage_rate'] == 0]
print(f"  數量: {len(no_service)} 個")

print("\n完全涵蓋的統計區:")
full_service = result[result['coverage_rate'] == 100]
print(f"  數量: {len(full_service)} 個")

# 顯示前10個涵蓋率最低的統計區
print("\n涵蓋率最低的前10個統計區:")
low_coverage = result.nsmallest(10, 'coverage_rate')[
    ['U_ID', 'TOWN', 'coverage_rate', 'coverage_level']
]
print(low_coverage.to_string(index=False))

print("\n" + "=" * 60)
print("分析完成!")
print("=" * 60)

# 輸出 CSV 方便查看
csv_path = OUTPUT_PATH.replace('.geojson', '.csv')
result_csv = result.drop(columns=['geometry'])
result_csv.to_csv(csv_path, index=False, encoding='utf-8-sig')
print(f"\n統計表格已儲存至: {csv_path}")

台北市最小統計區開放空間服務涵蓋率分析

[1/6] 讀取資料...
  ✓ 土地利用資料: 15529 筆
  ✓ 最小統計區: 11490 筆
  ✓ 土地利用座標系統: EPSG:3826
  ✓ 最小統計區座標系統: EPSG:3826

[2/6] 座標系統轉換至 WGS84...
  ✓ 土地利用資料已轉換
  ✓ 最小統計區已轉換

[3/6] 建立開放空間服務範圍...

  土地利用分區類型:
  共 250 種分區類型

  ✓ 篩選出 853 筆開放空間
  ✓ 開放空間類型: ['公園用地' '公園綠地' '公園廣場用地']

  【面積門檻篩選】
  設定最小面積門檻: 1000 平方公尺
  篩選前開放空間數量: 853
  面積統計:
    - 最小: 0.00 ㎡
    - 最大: 1366224.47 ㎡
    - 平均: 13498.99 ㎡
    - 中位數: 2184.87 ㎡

  ✓ 移除 188 個過小綠地 (< 1000㎡)
  ✓ 保留 665 個有效服務性開放空間
  ✓ 被移除的小綠地面積範圍: 0 - 994 ㎡

[4/6] 建立 300 公尺服務範圍緩衝區...
  ✓ 合併重疊的緩衝區...

[5/6] 計算每個統計區的服務涵蓋率...
  ✓ 計算完成!

[6/6] 轉換回 WGS84 並儲存結果...
  ✓ 結果已儲存至: taipei_open_space_coverage.geojson

分析結果摘要

總統計區數: 11490
平均涵蓋率: 87.22%
中位數涵蓋率: 100.00%
最高涵蓋率: 100.00%
最低涵蓋率: 0.00%

涵蓋率分級統計:
coverage_level
低(<25%)       328
中(25-50%)     241
高(50-75%)     312
極高(>75%)     9756
Name: count, dtype: int64

完全無服務的統計區:
  數量: 853 個

完全涵蓋的統計區:
  數量: 9170 個

涵蓋率最低的前10個統計區:
  U_ID TOWN  coverage_rate coverage_level
2153.0  內湖區            0.0            NaN
216

# filter green space dervice with area and ndvi

In [20]:
import geopandas as gpd
import pandas as pd
from shapely.ops import unary_union
import warnings
warnings.filterwarnings('ignore')


# ===== 設定參數 =====
LAND_USE_PATH = 'p113119-3_valid_geometry_3826.geojson'  # 土地利用資料路徑
STAT_AREA_PATH = 'STAT/113年12月臺北市統計區人口統計_最小統計區/tp_最小統計區.geojson'  # 最小統計區資料路徑
OUTPUT_PATH = 'taipei_open_space_coverage_ndvi.geojson'  # 輸出路徑

# HLS L30 NDVI 影像路徑 (選用)
NDVI_RASTER_PATH = 'ndvi_output/ndvi_result_fixed_2024185_wgs84.tif'  # 如: 'HLS_L30_NDVI_2024.tif' 或 None 表示不使用
NDVI_THRESHOLD = 0.35    # NDVI門檻值 (0.3-0.4 代表有植被)
NDVI_COVERAGE_MIN = 40   # 綠化覆蓋最低比例 (%)


# 條件性導入 rasterio (僅當使用 NDVI 時)
if 'NDVI_RASTER_PATH' in dir() and NDVI_RASTER_PATH is not None:
    try:
        import rasterio
        from rasterio.mask import mask
        import numpy as np
        NDVI_AVAILABLE = True
    except ImportError:
        print("警告: 未安裝 rasterio,將跳過 NDVI 篩選")
        print("安裝方式: pip install rasterio")
        NDVI_AVAILABLE = False
else:
    NDVI_AVAILABLE = False

# 多重服務範圍設定 (公尺)
BUFFER_DISTANCES = {
    'strict_300m': 300,    # 嚴格標準 (歐洲標準/日常步行)
    'standard_500m': 300,  # 標準服務範圍 (國際常用)
    'relaxed_800m': 300    # 寬鬆標準 (區域公園)
}

# 開放空間最小面積門檻 (平方公尺)
MIN_AREA_THRESHOLD = 1000  # 建議值: 500-2000㎡
# 說明:
# - 500㎡: 非常寬鬆,包含小型綠地
# - 1000㎡: 標準門檻,約0.1公頃
# - 2000㎡: 嚴格標準,確保有效服務功能

# 定義開放空間類型 (根據你的資料,請依實際狀況調整)
OPEN_SPACE_TYPES = [
    '公園',
    '公園用地',
    '公園綠地',
    '公園廣場用地',
    '兒童遊樂場',
    '兒童遊樂場用地',
    '廣場用地',
    '綠地',
    '體育場所用地',
    '運動場',
    '學校用地',
    # '社教用地',
    # '陽明山公園',  # 根據你的截圖
    # '人行步道(無遮蔽)',
    # '人行步道'
]

print("=" * 60)
print("台北市最小統計區開放空間服務涵蓋率分析")
print("=" * 60)

# ===== Step 1: 讀取資料 =====
print("\n[1/6] 讀取資料...")
land_use = gpd.read_file(LAND_USE_PATH)
stat_areas = gpd.read_file(STAT_AREA_PATH)

print(f"  ✓ 土地利用資料: {len(land_use)} 筆")
print(f"  ✓ 最小統計區: {len(stat_areas)} 筆")
print(f"  ✓ 土地利用座標系統: {land_use.crs}")
print(f"  ✓ 最小統計區座標系統: {stat_areas.crs}")

# ===== Step 2: 座標轉換 =====
print("\n[2/6] 座標系統轉換至 WGS84...")
if land_use.crs != 'EPSG:4326':
    land_use = land_use.to_crs('EPSG:4326')
    print(f"  ✓ 土地利用資料已轉換")

if stat_areas.crs != 'EPSG:4326':
    stat_areas = stat_areas.to_crs('EPSG:4326')
    print(f"  ✓ 最小統計區已轉換")

# 為了計算緩衝區(單位:公尺),暫時轉換至 TWD97 TM2
print("\n[3/6] 建立開放空間服務範圍...")
land_use_meter = land_use.to_crs('EPSG:3826')
stat_areas_meter = stat_areas.to_crs('EPSG:3826')

# ===== Step 3: 篩選開放空間 =====
# 檢查實際的使用分區欄位名稱
land_use_column = '使用分區'  # 根據你的截圖

print(f"\n  土地利用分區類型:")
unique_types = land_use[land_use_column].unique()
print(f"  共 {len(unique_types)} 種分區類型")

# 使用模糊比對找出開放空間
open_spaces_mask = land_use_meter[land_use_column].str.contains(
    # '|'.join(['公園', '綠地', '廣場', '遊樂', '步道', '體育', '運動']), 
    '|'.join(['公園', '綠地', '廣場', '遊樂', '步道', '體育', '運動']), 
    case=False, 
    na=False
)

open_spaces = land_use_meter[open_spaces_mask].copy()
print(f"\n  ✓ 篩選出 {len(open_spaces)} 筆開放空間")

if len(open_spaces) > 0:
    print(f"  ✓ 開放空間類型: {open_spaces[land_use_column].unique()[:10]}")

# ===== 面積篩選 =====
print(f"\n  【面積門檻篩選】")
print(f"  設定最小面積門檻: {MIN_AREA_THRESHOLD} 平方公尺")

# 計算每個開放空間的面積
open_spaces['area_sqm'] = open_spaces.geometry.area

# 面積統計
print(f"  篩選前開放空間數量: {len(open_spaces)}")
print(f"  面積統計:")
print(f"    - 最小: {open_spaces['area_sqm'].min():.2f} ㎡")
print(f"    - 最大: {open_spaces['area_sqm'].max():.2f} ㎡")
print(f"    - 平均: {open_spaces['area_sqm'].mean():.2f} ㎡")
print(f"    - 中位數: {open_spaces['area_sqm'].median():.2f} ㎡")

# 篩選符合面積門檻的開放空間
open_spaces_filtered = open_spaces[open_spaces['area_sqm'] >= MIN_AREA_THRESHOLD].copy()

small_spaces_count = len(open_spaces) - len(open_spaces_filtered)
print(f"\n  ✓ 移除 {small_spaces_count} 個過小綠地 (< {MIN_AREA_THRESHOLD}㎡)")
print(f"  ✓ 保留 {len(open_spaces_filtered)} 個有效服務性開放空間")

# 顯示被移除的小綠地面積範圍
if small_spaces_count > 0:
    small_spaces = open_spaces[open_spaces['area_sqm'] < MIN_AREA_THRESHOLD]
    print(f"  ✓ 被移除的小綠地面積範圍: {small_spaces['area_sqm'].min():.0f} - {small_spaces['area_sqm'].max():.0f} ㎡")

# 使用篩選後的開放空間繼續分析
open_spaces = open_spaces_filtered

# ===== NDVI 篩選 (選用) =====
if NDVI_AVAILABLE and NDVI_RASTER_PATH is not None:
    print(f"\n  【NDVI 綠化品質篩選】")
    print(f"  NDVI 影像: {NDVI_RASTER_PATH}")
    print(f"  NDVI 門檻: {NDVI_THRESHOLD}")
    print(f"  最低綠化覆蓋率: {NDVI_COVERAGE_MIN}%")
    
    try:
        # 開啟 NDVI 影像
        with rasterio.open(NDVI_RASTER_PATH) as src:
            # 確保座標系統一致
            if open_spaces.crs != src.crs:
                open_spaces_ndvi = open_spaces.to_crs(src.crs)
            else:
                open_spaces_ndvi = open_spaces.copy()
            
            # 計算每個開放空間的 NDVI 統計
            ndvi_stats = []
            
            for idx, row in open_spaces_ndvi.iterrows():
                try:
                    # 裁切 NDVI 影像
                    out_image, out_transform = mask(
                        src, 
                        [row.geometry], 
                        crop=True,
                        nodata=src.nodata
                    )
                    
                    # 提取 NDVI 值
                    ndvi_values = out_image[0]
                    
                    # 移除 nodata
                    valid_ndvi = ndvi_values[
                        (ndvi_values != src.nodata) & 
                        (~np.isnan(ndvi_values))
                    ]
                    
                    if len(valid_ndvi) > 0:
                        # 計算統計
                        mean_ndvi = np.mean(valid_ndvi)
                        max_ndvi = np.max(valid_ndvi)
                        
                        # 計算高 NDVI 像元比例
                        high_ndvi_ratio = (
                            np.sum(valid_ndvi >= NDVI_THRESHOLD) / 
                            len(valid_ndvi) * 100
                        )
                        
                        ndvi_stats.append({
                            'mean_ndvi': mean_ndvi,
                            'max_ndvi': max_ndvi,
                            'green_coverage': high_ndvi_ratio
                        })
                    else:
                        ndvi_stats.append({
                            'mean_ndvi': 0,
                            'max_ndvi': 0,
                            'green_coverage': 0
                        })
                        
                except Exception as e:
                    # 處理個別圖斑的錯誤
                    ndvi_stats.append({
                        'mean_ndvi': 0,
                        'max_ndvi': 0,
                        'green_coverage': 0
                    })
            
            # 將 NDVI 統計加入 GeoDataFrame
            ndvi_df = pd.DataFrame(ndvi_stats)
            open_spaces = pd.concat([
                open_spaces.reset_index(drop=True), 
                ndvi_df
            ], axis=1)
            
            # 篩選符合 NDVI 條件的開放空間
            print(f"\n  NDVI 統計:")
            print(f"    - 平均 NDVI: {open_spaces['mean_ndvi'].mean():.3f}")
            print(f"    - 平均綠化覆蓋率: {open_spaces['green_coverage'].mean():.1f}%")
            
            open_spaces_before_ndvi = len(open_spaces)
            open_spaces = open_spaces[
                (open_spaces['mean_ndvi'] >= NDVI_THRESHOLD) | 
                (open_spaces['green_coverage'] >= NDVI_COVERAGE_MIN)
            ].copy()
            
            removed_count = open_spaces_before_ndvi - len(open_spaces)
            print(f"\n  ✓ 移除 {removed_count} 個綠化不足的空間")
            print(f"  ✓ 保留 {len(open_spaces)} 個高品質綠地")
            
    except Exception as e:
        print(f"\n  ⚠️ NDVI 處理錯誤: {str(e)}")
        print(f"  ⚠️ 將略過 NDVI 篩選,繼續使用面積篩選結果")
else:
    print(f"\n  ℹ️ 未使用 NDVI 篩選 (NDVI_RASTER_PATH = {NDVI_RASTER_PATH})")
    # 不使用 NDVI 時,添加空欄位以保持一致性
    open_spaces['mean_ndvi'] = None
    open_spaces['max_ndvi'] = None
    open_spaces['green_coverage'] = None

## save open spaces
open_spaces.to_file("taipei_open_spaces_with_ndvi.geojson", driver="GeoJSON", encoding="utf-8")

# ===== Step 4: 建立多重緩衝區並計算涵蓋率 =====
print(f"\n[4/6] 建立多重服務範圍緩衝區...")

# 計算統計區總面積
stat_areas_meter['total_area'] = stat_areas_meter.geometry.area

# 對每個緩衝距離進行分析
for buffer_name, distance in BUFFER_DISTANCES.items():
    print(f"\n  >>> 分析 {buffer_name} ({distance}公尺) <<<")
    
    # 建立緩衝區
    buffer_geoms = open_spaces.geometry.buffer(distance)
    
    # 合併所有緩衝區
    merged_buffer = unary_union(buffer_geoms)
    
    # 計算服務涵蓋面積
    stat_areas_meter[f'service_area_{buffer_name}'] = stat_areas_meter.geometry.intersection(merged_buffer).area
    
    # 計算涵蓋率
    stat_areas_meter[f'coverage_{buffer_name}'] = (
        stat_areas_meter[f'service_area_{buffer_name}'] / 
        stat_areas_meter['total_area'] * 100
    ).round(2)
    
    print(f"  ✓ 平均涵蓋率: {stat_areas_meter[f'coverage_{buffer_name}'].mean():.2f}%")

# 建立綜合分級 (以標準500m為基準)
stat_areas_meter['coverage_level'] = pd.cut(
    stat_areas_meter['coverage_standard_500m'],
    bins=[0, 25, 50, 75, 100],
    labels=['低(<25%)', '中(25-50%)', '高(50-75%)', '極高(>75%)']
)

# 計算服務差距 (300m vs 500m)
stat_areas_meter['service_gap'] = (
    stat_areas_meter['coverage_standard_500m'] - 
    stat_areas_meter['coverage_strict_300m']
).round(2)

print(f"  ✓ 計算完成!")

# ===== Step 6: 轉換回 WGS84 並輸出 =====
print("\n[6/6] 轉換回 WGS84 並儲存結果...")

# 移除臨時幾何欄位,轉換回 WGS84
# result = stat_areas_meter.drop(columns=['service_geom']).to_crs('EPSG:4326')
# 移除臨時欄位,轉換回 WGS84
cols_to_drop = [col for col in stat_areas_meter.columns if col.startswith('service_area_')]
result = stat_areas_meter.drop(columns=cols_to_drop).to_crs('EPSG:4326')
# 確保幾何欄位是主幾何
result = result.set_geometry('geometry')

# 移除可能的空幾何
result = result[~result.geometry.is_empty]

# 儲存結果為 GeoJSON
result.to_file(OUTPUT_PATH, driver='GeoJSON', encoding='utf-8')

# 驗證輸出
print(f"  ✓ 輸出幾何類型: {result.geometry.geom_type.unique()}")

print(f"  ✓ 結果已儲存至: {OUTPUT_PATH}")

# ===== 統計摘要 =====
print("\n" + "=" * 60)
print("分析結果摘要")
print("=" * 60)

print(f"\n總統計區數: {len(result)}")

# 各距離標準的統計
for buffer_name, distance in BUFFER_DISTANCES.items():
    col_name = f'coverage_{buffer_name}'
    print(f"\n【{buffer_name.upper()} - {distance}公尺服務範圍】")
    print(f"  平均涵蓋率: {result[col_name].mean():.2f}%")
    print(f"  中位數涵蓋率: {result[col_name].median():.2f}%")
    print(f"  最高涵蓋率: {result[col_name].max():.2f}%")
    print(f"  最低涵蓋率: {result[col_name].min():.2f}%")
    
    # 無服務區域
    no_service = len(result[result[col_name] == 0])
    print(f"  完全無服務的統計區: {no_service} 個 ({no_service/len(result)*100:.1f}%)")
    
    # 完全涵蓋區域
    full_service = len(result[result[col_name] == 100])
    print(f"  完全涵蓋的統計區: {full_service} 個 ({full_service/len(result)*100:.1f}%)")

# 服務差距分析
print("\n【服務差距分析 (500m vs 300m)】")
print(f"  平均差距: {result['service_gap'].mean():.2f}%")
print(f"  最大差距: {result['service_gap'].max():.2f}%")
print(f"  差距>30%的統計區: {len(result[result['service_gap'] > 30])} 個")

print("\n涵蓋率分級統計 (以500m為基準):")
print(result['coverage_level'].value_counts().sort_index())

# 顯示涵蓋率最低的前10個統計區
print("\n【最需要改善的統計區 TOP 10 (300m標準)】")
low_coverage = result.nsmallest(10, 'coverage_strict_300m')[
    ['U_ID', 'TOWN', 'coverage_strict_300m', 'coverage_standard_500m', 'service_gap']
]
print(low_coverage.to_string(index=False))

print("\n" + "=" * 60)
print("分析完成!")
print("=" * 60)

# 輸出 CSV 方便查看
csv_path = OUTPUT_PATH.replace('.geojson', '.csv')
result_csv = result.drop(columns=['geometry'])
result_csv.to_csv(csv_path, index=False, encoding='utf-8-sig')
print(f"\n統計表格已儲存至: {csv_path}")

# 輸出開放空間清單 (含面積資訊)
open_spaces_csv = OUTPUT_PATH.replace('.geojson', '_open_spaces_list.csv')
open_spaces_export = open_spaces[[land_use_column, 'area_sqm']].copy()
open_spaces_export = open_spaces_export.sort_values('area_sqm', ascending=False)
open_spaces_export.to_csv(open_spaces_csv, index=False, encoding='utf-8-sig')
print(f"開放空間清單已儲存至: {open_spaces_csv}")

台北市最小統計區開放空間服務涵蓋率分析

[1/6] 讀取資料...
  ✓ 土地利用資料: 15529 筆
  ✓ 最小統計區: 11490 筆
  ✓ 土地利用座標系統: EPSG:3826
  ✓ 最小統計區座標系統: EPSG:3826

[2/6] 座標系統轉換至 WGS84...
  ✓ 土地利用資料已轉換
  ✓ 最小統計區已轉換

[3/6] 建立開放空間服務範圍...

  土地利用分區類型:
  共 250 種分區類型

  ✓ 篩選出 1526 筆開放空間
  ✓ 開放空間類型: ['公園用地' '廣場用地' '人行步道用地' '綠地用地' '交通廣場用地' '體育場用地' '兒童遊樂場用地' '公園綠地'
 '市民運動中心用地' '公園廣場用地']

  【面積門檻篩選】
  設定最小面積門檻: 1000 平方公尺
  篩選前開放空間數量: 1526
  面積統計:
    - 最小: 0.00 ㎡
    - 最大: 1366224.47 ㎡
    - 平均: 8966.23 ㎡
    - 中位數: 1520.97 ㎡

  ✓ 移除 541 個過小綠地 (< 1000㎡)
  ✓ 保留 985 個有效服務性開放空間
  ✓ 被移除的小綠地面積範圍: 0 - 998 ㎡

  【NDVI 綠化品質篩選】
  NDVI 影像: ndvi_output/ndvi_result_fixed_2024185_wgs84.tif
  NDVI 門檻: 0.35
  最低綠化覆蓋率: 40%

  NDVI 統計:
    - 平均 NDVI: 0.506
    - 平均綠化覆蓋率: 78.0%

  ✓ 移除 181 個綠化不足的空間
  ✓ 保留 804 個高品質綠地

[4/6] 建立多重服務範圍緩衝區...

  >>> 分析 strict_300m (300公尺) <<<
  ✓ 平均涵蓋率: 89.54%

  >>> 分析 standard_500m (300公尺) <<<
  ✓ 平均涵蓋率: 89.54%

  >>> 分析 relaxed_800m (300公尺) <<<
  ✓ 平均涵蓋率: 89.54%
  ✓ 計算完成!

[6/6] 轉換回 WGS84 並儲存結果...
  ✓ 輸出幾何類型: ['MultiPolyg

In [19]:
open_spaces.to_file("taipei_open_spaces_with_ndvi.geojson", driver="GeoJSON", encoding="utf-8")